# Análise e exploração e tratamento de dados com SQL e Duckdb

Nesse Notebook venho apresentar um exemplo de análise e tratamento de dados de um arquivo CSV utilizando Duckdb.

Como exemplo criei os seguintes indicadores como objeto de estudo.

### Indicadores
1. Qual é a quantidade e o percentual de candidatos formados e não formados no ensino médio?
2. Qual é a distribuição de participação dos candidatos por:
    - Estado;
    - Tipo de escola (Pública e Privada)
    - Sexo
    - Cor/raça
    - Faixa Etária

## Análise dos dados

Realizei o download do dataset ENEM 2023 no site do governo federal https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/enem

Após a leitura do docionário de dados disponível no path DICIONÁRIO/Dicionário_Microdados_Enem_2023.xlsx 
tentei realizar a leitura do arquivo e cheguei na seguinte solução para fazer a leitura pois enfrentei os seguintes desafios:

- O dataset MICRODADOS_ENEM_2023 está no formato CSV encoding iso-8859 e possui mais de 1GB. O que é necessário realizar o tratamento do arquivo por partes otimizar a utilização dos recursos.
- O dados de algumas colunas possuem um DE-PARA com a regra descrita no dicionário de dados. Para facilitar a análise dos dados será necessário realizar o DE-PARA dos valores no dataset

## Soluções encontrada

Como opção pessoal eu prefiro tratar os dados com SQL em um banco relacional do que ficar tratando lote de dados com pandas. 

Para não ter que utilizar sistemas SGBDs ou Docker resolvi testar a biblioteca Duckdb que está ganhando espaço no mercado.

Pois, ela me permite salvar dados em um arquivo binário realizar instruções SQL de maneira fácil e rápida.

In [1]:
import duckdb



In [2]:
path = r'DATA/DADOS/MICRODADOS_ENEM_2023.csv'

columns = [
    'NU_ANO', 
    'TP_FAIXA_ETARIA', 
    'TP_SEXO', 
    'TP_COR_RACA', 
    'TP_ANO_CONCLUIU', 
    'TP_ESCOLA', 
    'TP_ENSINO', 
    'SG_UF_ESC', 
    'TP_ST_CONCLUSAO'
]


### Criando arquivo de banco de dados

In [3]:
conn = duckdb.connect('dataset_enem.db')


### Carregando tabela no duckdb com os lotes de dados extraídos do arquivo CSV

Tempo de execução 1.5s


In [4]:
tabela = conn.execute(
    f"""
        CREATE TABLE IF NOT EXISTS dataset_enem AS 
        SELECT 
            {', '.join(columns)}
        FROM read_csv_auto('{path}', delim=';', header=True, ignore_errors=true)
    """
)

conn.sql("SELECT * from dataset_enem")

┌────────┬─────────────────┬─────────┬─────────────┬─────────────────┬───────────┬───────────┬───────────┬─────────────────┐
│ NU_ANO │ TP_FAIXA_ETARIA │ TP_SEXO │ TP_COR_RACA │ TP_ANO_CONCLUIU │ TP_ESCOLA │ TP_ENSINO │ SG_UF_ESC │ TP_ST_CONCLUSAO │
│ int64  │      int64      │ varchar │    int64    │      int64      │   int64   │   int64   │  varchar  │      int64      │
├────────┼─────────────────┼─────────┼─────────────┼─────────────────┼───────────┼───────────┼───────────┼─────────────────┤
│   2023 │              14 │ M       │           1 │              17 │         1 │      NULL │ NULL      │               1 │
│   2023 │              12 │ M       │           1 │              16 │         1 │      NULL │ NULL      │               1 │
│   2023 │               6 │ F       │           1 │               0 │         1 │      NULL │ NULL      │               1 │
│   2023 │               2 │ F       │           3 │               0 │         2 │         1 │ CE        │               2 │


### Criando tabela auxiliar com dados tratados utilizando SQL

In [5]:
conn.sql("""
CREATE OR REPLACE TABLE dataset_enem_tratado AS
    SELECT 
        CASE
            WHEN TP_FAIXA_ETARIA = 1 THEN 'Menor de 17 anos'
            WHEN TP_FAIXA_ETARIA = 2 THEN '17 anos'
            WHEN TP_FAIXA_ETARIA = 3 THEN '18 anos'
            WHEN TP_FAIXA_ETARIA = 4 THEN '19 anos'
            WHEN TP_FAIXA_ETARIA = 5 THEN '20 anos'
            WHEN TP_FAIXA_ETARIA = 6 THEN '21 anos'
            WHEN TP_FAIXA_ETARIA = 7 THEN '22 anos'
            WHEN TP_FAIXA_ETARIA = 8 THEN '23 anos'
            WHEN TP_FAIXA_ETARIA = 9 THEN '24 anos'
            WHEN TP_FAIXA_ETARIA = 10 THEN '25 anos'
            WHEN TP_FAIXA_ETARIA = 11 THEN 'Entre 26 e 30 anos'
            WHEN TP_FAIXA_ETARIA = 12 THEN 'Entre 31 e 35 anos'
            WHEN TP_FAIXA_ETARIA = 13 THEN 'Entre 36 e 40 anos'
            WHEN TP_FAIXA_ETARIA = 14 THEN 'Entre 41 e 45 anos'
            WHEN TP_FAIXA_ETARIA = 15 THEN 'Entre 46 e 50 anos'
            WHEN TP_FAIXA_ETARIA = 16 THEN 'Entre 51 e 55 anos'
            WHEN TP_FAIXA_ETARIA = 17 THEN 'Entre 56 e 60 anos'
            WHEN TP_FAIXA_ETARIA = 18 THEN 'Entre 61 e 65 anos'
            WHEN TP_FAIXA_ETARIA = 19 THEN 'Entre 66 e 70 anos'
            WHEN TP_FAIXA_ETARIA = 20 THEN 'Maior de 70 anos'
            ELSE 'Não Informado'
        END NO_FAIXA_ETARIA
        , CASE
            WHEN TP_SEXO = 'M' THEN 'Masculino'
            WHEN TP_SEXO = 'F' THEN 'Feminino'
            ELSE 'Não Informado'
        END NO_SEXO
        , CASE
            WHEN TP_COR_RACA = '0' THEN 'Não declarado'
            WHEN TP_COR_RACA = '1' THEN 'Branca'
            WHEN TP_COR_RACA = '2' THEN 'Preta'
            WHEN TP_COR_RACA = '3' THEN 'Parda'
            WHEN TP_COR_RACA = '4' THEN 'Amarela'
            WHEN TP_COR_RACA = '5' THEN 'Indígena '
            WHEN TP_COR_RACA = '6' THEN 'Não dispõe da informação'
            ELSE 'Não Informado'
        END NO_ETNIA
        , CASE
            WHEN TP_ST_CONCLUSAO = 1 THEN 'Já concluí o Ensino Médio'
            WHEN TP_ST_CONCLUSAO = 2 THEN 'Estou cursando e concluirei o Ensino Médio em 2023'
            WHEN TP_ST_CONCLUSAO = 3 THEN 'Estou cursando e concluirei o Ensino Médio após 2023'
            WHEN TP_ST_CONCLUSAO = 4 THEN 'Não concluí e não estou cursando o Ensino Médio'
            ELSE 'Não Informado'
        END DS_CONLUSAO_ENSINO
        , CASE
            WHEN TP_ANO_CONCLUIU = 0 THEN 'Não informado'
            WHEN TP_ANO_CONCLUIU = 1 THEN '2022'
            WHEN TP_ANO_CONCLUIU = 2 THEN '2021'
            WHEN TP_ANO_CONCLUIU = 3 THEN '2020'
            WHEN TP_ANO_CONCLUIU = 4 THEN '2019'
            WHEN TP_ANO_CONCLUIU = 5 THEN '2018'
            WHEN TP_ANO_CONCLUIU = 6 THEN '2017'
            WHEN TP_ANO_CONCLUIU = 7 THEN '2016'
            WHEN TP_ANO_CONCLUIU = 8 THEN '2015'
            WHEN TP_ANO_CONCLUIU = 9 THEN '2014'
            WHEN TP_ANO_CONCLUIU = 10 THEN '2013'
            WHEN TP_ANO_CONCLUIU = 11 THEN '2012'
            WHEN TP_ANO_CONCLUIU = 12 THEN '2011'
            WHEN TP_ANO_CONCLUIU = 13 THEN '2010'
            WHEN TP_ANO_CONCLUIU = 14 THEN '2009'
            WHEN TP_ANO_CONCLUIU = 15 THEN '2008'
            WHEN TP_ANO_CONCLUIU = 16 THEN '2007'
            WHEN TP_ANO_CONCLUIU = 17 THEN 'Antes de 2007'
            ELSE 'Não Informado'
        END NU_ANO_CONCLUSAO
        , CASE
            WHEN TP_ESCOLA = 1 THEN 'Não Respondeu'
            WHEN TP_ESCOLA = 2 THEN 'Pública'
            WHEN TP_ESCOLA = 3 THEN 'Privada'
            ELSE 'Não Informado'
        END NO_TIPO_ESCOLA
        , CASE
            WHEN TP_ENSINO = 1 THEN 'Ensino Regular'
            WHEN TP_ENSINO = 2 THEN 'Educação Especial - Modalidade Substitutiva'
            ELSE 'Não Informado'
        END NO_TIPO_ENSINO
        , COALESCE(de.SG_UF_ESC, 'Não Informado') NO_UF
        , COALESCE(de.NU_ANO, 2023) NU_ANO
    FROM dataset_enem de
""")
conn.sql("SELECT * FROM dataset_enem_tratado")

┌────────────────────┬───────────┬───────────────┬────────────────────────────────────────────────────┬──────────────────┬────────────────┬────────────────┬───────────────┬────────┐
│  NO_FAIXA_ETARIA   │  NO_SEXO  │   NO_ETNIA    │                 DS_CONLUSAO_ENSINO                 │ NU_ANO_CONCLUSAO │ NO_TIPO_ESCOLA │ NO_TIPO_ENSINO │     NO_UF     │ NU_ANO │
│      varchar       │  varchar  │    varchar    │                      varchar                       │     varchar      │    varchar     │    varchar     │    varchar    │ int64  │
├────────────────────┼───────────┼───────────────┼────────────────────────────────────────────────────┼──────────────────┼────────────────┼────────────────┼───────────────┼────────┤
│ Entre 41 e 45 anos │ Masculino │ Branca        │ Já concluí o Ensino Médio                          │ Antes de 2007    │ Não Respondeu  │ Não Informado  │ Não Informado │   2023 │
│ Entre 31 e 35 anos │ Masculino │ Branca        │ Já concluí o Ensino Médio              

### Qual é a quantidade e o percentual de candidatos formados e não formados no ensino médio?

In [6]:
conn.sql("""
    SELECT 
        DS_CONLUSAO_ENSINO
        , COUNT(1) QT_CANDIDATOS 
        , (COUNT(1) / (SELECT COUNT(1) FROM dataset_enem_tratado)*100) QT_PERCENTUAL_TOTAL
    FROM dataset_enem_tratado
    GROUP BY DS_CONLUSAO_ENSINO 
""")

┌──────────────────────────────────────────────────────┬───────────────┬─────────────────────┐
│                  DS_CONLUSAO_ENSINO                  │ QT_CANDIDATOS │ QT_PERCENTUAL_TOTAL │
│                       varchar                        │     int64     │       double        │
├──────────────────────────────────────────────────────┼───────────────┼─────────────────────┤
│ Estou cursando e concluirei o Ensino Médio em 2023   │       1401164 │   35.61718423316993 │
│ Já concluí o Ensino Médio                            │       1895301 │   48.17800406969576 │
│ Não concluí e não estou cursando o Ensino Médio      │         17423 │ 0.44288762835365425 │
│ Estou cursando e concluirei o Ensino Médio após 2023 │        620067 │  15.761924068780656 │
└──────────────────────────────────────────────────────┴───────────────┴─────────────────────┘

### Qual é a distribuição de participação dos candidatos por:
    - Estado;
    - Tipo de escola (Pública e Privada);
    - Sexo;
    - Cor/raça;
    - Faixa Etária.

In [7]:
conn.sql("""
    SELECT 
        NO_UF
        , NO_TIPO_ESCOLA
        , NO_ETNIA
        , NO_SEXO
        , NO_FAIXA_ETARIA
        , COUNT(1) QT_CANDIDATOS 
    FROM dataset_enem_tratado
    WHERE NO_UF <> 'Não Informado'
    GROUP BY NO_UF
        , NO_TIPO_ESCOLA
        , NO_ETNIA
        , NO_SEXO
        , NO_FAIXA_ETARIA 
""")

┌─────────┬────────────────┬───────────────┬───────────┬────────────────────┬───────────────┐
│  NO_UF  │ NO_TIPO_ESCOLA │   NO_ETNIA    │  NO_SEXO  │  NO_FAIXA_ETARIA   │ QT_CANDIDATOS │
│ varchar │    varchar     │    varchar    │  varchar  │      varchar       │     int64     │
├─────────┼────────────────┼───────────────┼───────────┼────────────────────┼───────────────┤
│ SP      │ Pública        │ Branca        │ Feminino  │ 18 anos            │         23247 │
│ DF      │ Privada        │ Branca        │ Feminino  │ 18 anos            │          1030 │
│ SC      │ Pública        │ Branca        │ Feminino  │ 18 anos            │          6544 │
│ CE      │ Pública        │ Branca        │ Feminino  │ 18 anos            │          4849 │
│ RN      │ Privada        │ Parda         │ Masculino │ 18 anos            │           309 │
│ SP      │ Privada        │ Branca        │ Feminino  │ 18 anos            │         10676 │
│ PR      │ Privada        │ Branca        │ Feminino  │ 17 